In [27]:
# 학습셋 파일 경로
train_path = '/content/drive/MyDrive/DKU_DATA/train_data.csv'

# 테스트셋 파일 경로
test_path = '/content/drive/MyDrive/DKU_DATA/test_data.csv'

# 1. 설정 및 라이브러리 호출

In [1]:
!pip install catboost #catboost 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 10.8 MB/s eta 0:00:00


In [28]:
import pandas as pd
import numpy as np
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import random
import os
from catboost import CatBoostClassifier
from imblearn.over_sampling import ADASYN
import pickle

# 2. 재실인원 0 or 1 이진분류

## a. Train 데이터셋

In [29]:
# CSV 파일을 DataFrame으로 읽어오기 (인코딩 방식 지정)
bin_train_df = pd.read_csv(train_path, encoding='cp949')

#0이 아닌 컬럼은 모두 1로 변환, 이진분류를 위해서.
bin_train_df.loc[bin_train_df['재실인원'] != 0, '재실인원'] = 1

# null 값을 포함하는 행(row) 제거
bin_train_df = bin_train_df.dropna()

# 중복된 행(row) 제거
bin_train_df = bin_train_df.drop_duplicates()

# regdate 컬럼을 datetime 형식으로 변환 후 시간을 반올림하여 0~23 사이의 값으로 변환
bin_train_df['regdate'] = pd.to_datetime(bin_train_df['regdate'], format='%Y-%m-%d %H:%M:%S')
bin_train_df['regdate'] = bin_train_df['regdate'].apply(lambda x: (x + datetime.timedelta(minutes=30)).hour % 24)

# ADASYN 오버 샘플링 적용
adasyn = ADASYN(random_state=28,sampling_strategy='auto')
X_oversampled, y_oversampled = adasyn.fit_resample(bin_train_df.drop('재실인원', axis=1), bin_train_df['재실인원'])

# 새로운 오버샘플링된 데이터프레임 생성
df_oversampled = pd.DataFrame(X_oversampled, columns=bin_train_df.drop('재실인원', axis=1).columns)
df_oversampled['재실인원'] = y_oversampled

bin_train_df = df_oversampled

# 최종 선택된 컬럼
selected_columns = ['temp', 'humi', 'co2', 'eco2','재실인원']

# 최종 학습 데이터
bin_train_data = bin_train_df[selected_columns]

# 결과 출력
print(bin_train_data)


             temp       humi         co2         eco2  재실인원
0       21.200000  34.000000  414.000000  2845.000000     1
1       21.200000  34.000000  411.000000  2845.000000     1
2       21.200000  34.000000  409.000000  2845.000000     1
3       21.200000  34.000000  409.000000  2829.000000     1
4       21.200000  34.000000  415.000000  2829.000000     1
...           ...        ...         ...          ...   ...
359460  17.600000  23.000000  456.221236   801.715554     0
359461  17.600000  23.000000  461.893297   793.000000     0
359462  16.975784  18.917189  401.082811   486.000000     0
359463  17.009045  18.943153  400.886306   486.000000     0
359464  17.100000  19.000000  401.985427   481.072864     0

[359465 rows x 5 columns]


## b. Test 데이터셋

In [30]:
# CSV 파일을 DataFrame으로 읽어오기 (인코딩 방식 지정)
bin_test_df = pd.read_csv(test_path, encoding='cp949')

# 0이 아닌 컬럼은 모두 1로 변환, 이진분류를 위해서.
bin_test_df.loc[bin_test_df['재실인원'] != 0, '재실인원'] = 1

# regdate 컬럼을 datetime 형식으로 변환 후 시간을 반올림하여 0~23 사이의 값으로 변환
bin_test_df['regdate'] = pd.to_datetime(bin_test_df['regdate'], format='%Y-%m-%d %H:%M:%S')
bin_test_df['regdate'] = bin_test_df['regdate'].apply(lambda x: (x + datetime.timedelta(minutes=30)).hour % 24)

# bin_train_data 데이터와 동일한 컬럼 사용, 최종 학습데이터
bin_test_df = bin_test_df[bin_train_data.columns]

# train_test_data에서 각 컬럼의 최빈값 계산
mode_values = bin_train_data.mode().iloc[0]

# test_test_data의 결측치를 최빈값으로 채우기
bin_test_data = bin_test_df.fillna(mode_values)

print(bin_test_data)

        temp  humi    co2   eco2  재실인원
0       19.4  31.0  422.0  463.0     0
1       19.4  31.0  418.0  473.0     0
2       19.4  31.0  419.0  461.0     0
3       19.4  31.0  433.0  473.0     0
4       19.4  31.0  434.0  473.0     0
...      ...   ...    ...    ...   ...
84995  107.3   0.0  462.0  534.0     0
84996  107.3   0.0  462.0  534.0     0
84997  107.3   0.0  467.0  534.0     0
84998  -29.7   0.0  466.0  534.0     0
84999  107.3   0.0  476.0  534.0     0

[85000 rows x 5 columns]


## c. 모델 학습 및 저장

In [31]:
# 학습 데이터셋
bin_y_train = bin_train_data['재실인원']
bin_X_train = bin_train_data.drop('재실인원', axis=1)

# 테스트 데이터셋 
bin_y_test = bin_test_data['재실인원']
bin_X_test = bin_test_data.drop('재실인원', axis=1)

#GPU 및 기타 씨드 고정
def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os


my_seed = 70
my_seed_everywhere(my_seed)


# 이진분류 모델 파라미터
params = {'iterations': 127, 
          'learning_rate': 0.0005941239405009986, 
          'depth': 4, 
          'border_count': 42, 
          'l2_leaf_reg': 7.849989300833608e-06, 
          'bagging_temperature': 33.75576593022031}


#GPU를 사용한 CatBoost
bin_model = CatBoostClassifier(
    **params,
    random_seed=44,
    loss_function='Logloss',
    eval_metric='Accuracy',
    verbose=0,
    task_type='GPU')


# 모델 훈련
bin_model.fit(bin_X_train, bin_y_train)

# 예측 수행
y_pred_prob = bin_model.predict_proba(bin_X_test)[:, 1]  # 클래스 1의 예측 확률을 가져옴
y_pred = np.where(y_pred_prob >= 0.5, 1, 0)  # 예측 확률을 기준으로 이진 예측으로 변환

# 정확도 계산
accuracy = accuracy_score(bin_y_test, y_pred)
print("Accuracy:", accuracy)

# ROC AUC 계산
roc_auc = roc_auc_score(bin_y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.9210352941176471
ROC AUC: 0.95920093019517


In [32]:
# 모델 저장
with open('bin_classifier.pkl', 'wb') as f:
    pickle.dump(bin_model, f)

# 3.  재실인원 1,2,3,4,5,6 분류

## a. Train 데이터셋

In [33]:
# CSV 파일을 DataFrame으로 읽어오기 (인코딩 방식 지정)
mul_train_df = pd.read_csv(train_path, encoding='cp949')

# null 값을 포함하는 행(row) 제거
mul_train_df = mul_train_df.dropna()

# 중복된 행(row) 제거
mul_train_df = mul_train_df.drop_duplicates()

# regdate 컬럼을 datetime 형식으로 변환 후 시간을 반올림하여 0~23 사이의 값으로 변환
mul_train_df['regdate'] = pd.to_datetime(mul_train_df['regdate'], format='%Y-%m-%d %H:%M:%S')
mul_train_df['regdate'] = mul_train_df['regdate'].apply(lambda x: (x + datetime.timedelta(minutes=30)).hour % 24)

# 클래스 5,6 추출
df_class5 = mul_train_df[mul_train_df['재실인원'] == 5]
df_class6 = mul_train_df[mul_train_df['재실인원'] == 6]

# 클래스 5,6 하나의 DataFrame에 모두 저장
df_combined = pd.concat([df_class5, df_class6], ignore_index=True)

# 재실인원이 0,1,2,3,4인 데이터만 남김
mul_train_df = mul_train_df[mul_train_df['재실인원'].isin([0,1,2,3,4])]

# ADASYN 오버샘플링 적용(0,1,2,3,4만 적용)
adasyn = ADASYN(random_state=28,sampling_strategy='all')
X_oversampled, y_oversampled = adasyn.fit_resample(mul_train_df.drop('재실인원', axis=1), mul_train_df['재실인원'])

# 새로운 오버샘플링된 데이터프레임 생성
df_oversampled = pd.DataFrame(X_oversampled, columns=mul_train_df.drop('재실인원', axis=1).columns)
df_oversampled['재실인원'] = y_oversampled

# 오버샘플링된 데이터프레임을 df로 명명
mul_train_df = df_oversampled

# 재실인원이 1,2,3,4인 데이터만 남기기(0인 데이터들 버리기)
mul_train_df = mul_train_df[mul_train_df['재실인원'].isin([1,2,3,4])]

# 앞서 저장했던 데이터프레임과 결합하여 재실인원이 1,2,3,4,5,6 인 데이터만 남기기
mul_train_df = pd.concat([mul_train_df,df_combined], ignore_index=True)

#최종 선택 컬럼
selected_columns = ['temp', 
                    'co2', 
                    '재실인원']

#최종 학습 데이터
mul_train_data = mul_train_df[selected_columns]

# 결과 출력
print(mul_train_data)

        temp     co2  재실인원
0       21.2   414.0     1
1       21.2   411.0     1
2       21.2   409.0     1
3       21.2   409.0     1
4       21.2   415.0     1
...      ...     ...   ...
518571  21.8   907.0     6
518572  22.0  1013.0     6
518573  22.1  1027.0     6
518574  22.1  1055.0     6
518575  22.1  1034.0     6

[518576 rows x 3 columns]


## b. Test 데이터셋

In [34]:
# CSV 파일을 DataFrame으로 읽어오기 (인코딩 방식 지정)
mul_test_df = pd.read_csv(test_path, encoding='cp949')

# regdate 컬럼을 datetime 형식으로 변환 후 시간을 반올림하여 0~23 사이의 값으로 변환
mul_test_df['regdate'] = pd.to_datetime(mul_test_df['regdate'], format='%Y-%m-%d %H:%M:%S')
mul_test_df['regdate'] = mul_test_df['regdate'].apply(lambda x: (x + datetime.timedelta(minutes=30)).hour % 24)

# 학습 데이터와 동일한 컬럼을 가짐, 최종 테스트 데이터
mul_test_df = mul_test_df[mul_train_data.columns]

# 학습 데이터에서 각 컬럼의 최빈값 계산
mode_values = mul_train_data.mode().iloc[0]

# 테스트 데이터의 결측치를 학습 데이터의 최빈값으로 채우기
mul_test_data = mul_test_df.fillna(mode_values)

# 결과 출력
print(mul_test_data)

        temp    co2  재실인원
0       19.4  422.0     0
1       19.4  418.0     0
2       19.4  419.0     0
3       19.4  433.0     0
4       19.4  434.0     0
...      ...    ...   ...
84995  107.3  462.0     0
84996  107.3  462.0     0
84997  107.3  467.0     0
84998  -29.7  466.0     0
84999  107.3  476.0     0

[85000 rows x 3 columns]


## c. 모델 학습 및 저장

In [35]:
# 학습 데이터셋
mul_y_train = mul_train_data['재실인원']
mul_X_train = mul_train_data.drop('재실인원', axis=1)


# 테스트 데이터셋
mul_y_test = mul_test_data['재실인원']
mul_X_test = mul_test_data.drop('재실인원', axis=1)


#optuna를 통해 얻은 파라미터
best_params = {'iterations': 623, 
               'learning_rate': 0.04515702617353955, 
               'depth': 4, 'border_count': 116, 
               'l2_leaf_reg': 0.00013388785139921037, 
               'bagging_temperature': 0.014332596608738344}


#모델
mul_model = CatBoostClassifier(task_type='GPU', 
                           random_state=26, 
                           loss_function='MultiClass', 
                           verbose=0, 
                           **best_params)

#모델 학습
mul_model.fit(mul_X_train, mul_y_train)

# 테스트 데이터셋에서 예측값 생성(1,2,3,4)
y_pred_temp = mul_model.predict(mul_X_test)
y_pred_temp = np.ravel(y_pred_temp)

# 최종 병합
# 앞서 0과 1로 이진분류한 결과를 복사
y_fin = np.copy(y_pred)

# 1로 분류된 데이터들만 새롭게 분류된 결과로 대체
y_fin[y_fin == 1] = y_pred_temp[y_fin == 1]


# 예측 결과 평가
# 정확도 계산
accuracy = accuracy_score(mul_y_test, y_fin)

# macro-F1 점수 계산
macro_f1 = f1_score(mul_y_test, y_fin, average='macro')

# weighted-F1 점수 계산
weighted_f1 = f1_score(mul_y_test, y_fin, average='weighted')

# 최종 결과 출력
print("=================================")
print()
print('Accuracy:', accuracy)
print('Macro-F1:', macro_f1)
print('Weighted-F1:', weighted_f1)
print()
print("=================================")


Accuracy: 0.7920235294117647
Macro-F1: 0.4693548497958332
Weighted-F1: 0.7964153722883529



In [36]:
# 모델 저장
with open('multi_classifier.pkl', 'wb') as f:
    pickle.dump(mul_model, f)

# 4. 최종 결과 출력

In [37]:
# 결과 출력
print("=================================")
print()
print('Accuracy:', accuracy)
print('Macro-F1:', macro_f1)
print('Weighted-F1:', weighted_f1)
print()
print("=================================")


Accuracy: 0.7920235294117647
Macro-F1: 0.4693548497958332
Weighted-F1: 0.7964153722883529

